<a href="https://colab.research.google.com/github/chutki26/dissertation-newscrawler/blob/main/DetectingActiveVsPassiveVoice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U spacy[cuda-autodetect]
!python -m spacy download en_core_web_trf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 457.4/457.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.3/236.3 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 735.6/735.6 kB 20.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 69.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 54.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import json
from tqdm.auto import tqdm
import re
import spacy
from spacy.matcher import Matcher

In [ ]:
start_file_path = '/content/drive/My Drive/articles'

In [ ]:
df = pd.read_pickle(f'{start_file_path}/relevant_articles/df_relevant_articles.pkl')

In [ ]:
df['source_domain'].value_counts()

,count
source_domain,
theguardian.com,5543
independent.co.uk,3053
hindustantimes.com,2187
news18.com,1923
apnews.com,1903
dailymail.co.uk,1697
indianexpress.com,1380
bbc.com,1317
cnn.com,1172


In [ ]:
spacy.prefer_gpu()
nlp = spacy.load('en_core_web_trf')

In [ ]:
# create rules : taken from https://stackoverflow.com/questions/74528441/detect-passive-or-active-sentence-from-text first answer

passive_rules = [
    [{'DEP': 'nsubjpass'}, {'DEP': 'aux', 'OP': '*'}, {'DEP': 'auxpass'}, {'TAG': 'VBN'}],
    [{'DEP': 'nsubjpass'}, {'DEP': 'aux', 'OP': '*'}, {'DEP': 'auxpass'}, {'TAG': 'VBZ'}],
    [{'DEP': 'nsubjpass'}, {'DEP': 'aux', 'OP': '*'}, {'DEP': 'auxpass'}, {'TAG': 'RB'}, {'TAG': 'VBN'}],
]

active_rules = [
    [{'DEP': 'nsubj'}, {'TAG': 'VBD', 'DEP': 'ROOT'}],
    [{'DEP': 'nsubj'}, {'TAG': 'VBP'}, {'TAG': 'VBG', 'OP': '!'}],
    [{'DEP': 'nsubj'}, {'DEP': 'aux', 'OP': '*'}, {'TAG': 'VB'}],
    [{'DEP': 'nsubj'}, {'DEP': 'aux', 'OP': '*'}, {'TAG': 'VBG'}],
    [{'DEP': 'nsubj'}, {'TAG': 'RB', 'OP': '*'}, {'TAG': 'VBG'}],
    [{'DEP': 'nsubj'}, {'TAG': 'RB', 'OP': '*'}, {'TAG': 'VBZ'}],
    [{'DEP': 'nsubj'}, {'TAG': 'RB', 'OP': '+'}, {'TAG': 'VBD'}],
]



In [ ]:
matcher = Matcher(nlp.vocab)  # Init. the matcher with a vocab (note matcher vocab must share same vocab with docs)
matcher.add('Passive',  passive_rules)  # Add passive rules to matcher
matcher.add('Active', active_rules)  # Add active rules to matcher

In [ ]:
df_news = df[df['source_domain'] == 'india.com'] # change based on what news source currently being analysed

In [ ]:
passive_sents = []
active_sents = []

for i in tqdm(range(len(df_news))):

  entire_doc = nlp(df_news.iloc[i]['full_text'])
  text = [sent.text for sent in entire_doc.sents] # create sentences -> split(".") doesn't work for things like U.N., U.S. etc.

  for sentence in text:
      doc = nlp(sentence)  # Process text with spaCy model
      matches = matcher(doc)  # Get matches
      if len(matches) > 0:
          for match_id, start, end in matches:
              string_id = nlp.vocab.strings[match_id]
              span = doc[start:end]  # the matched span
              if string_id == 'Passive':
                  passive_sents.append({'sentence':sentence, 'span':span})
              elif string_id == 'Active':
                  active_sents.append({'sentence':sentence, 'span':span})

passive_df = pd.DataFrame(passive_sents)
active_df = pd.DataFrame(active_sents)

passive_df.to_csv(f'{start_file_path}/active_vs_passive_voice/india_passive_sentences.csv', index=False) # change these to the name of the newssite
active_df.to_csv(f'{start_file_path}/active_vs_passive_voice/india_active_sentences.csv', index=False)